In [1]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
from pudl import ferc1, pudl, models, models_ferc1, settings, constants
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
%load_ext autoreload
%autoreload 1

In [ ]:
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.models
%aimport pudl.models_ferc1
%aimport pudl.eia923
%aimport pudl.constants

In [ ]:
ferc1_engine = pudl.ferc1.db_connect_ferc1()
pudl.ferc1.init_db(refyear=2015, years=range(2004,2016), def_db=True)

In [ ]:
ferc1_respondents = pd.read_sql('''SELECT respondent_id, respondent_name FROM f1_respondent_id''', ferc1_engine)
ferc1_steam_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, tot_capacity
                                    FROM f1_steam''', ferc1_engine)
ferc1_small_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, kind_of_fuel, capacity_rating
                                    FROM f1_gnrt_plant''', ferc1_engine)
ferc1_hydro_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_hydro''', ferc1_engine)
ferc1_pumped_storage_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_pumped_storage''', ferc1_engine)

In [ ]:
ferc1_all_plants = pd.concat([ferc1_steam_plants, ferc1_small_plants, ferc1_hydro_plants, ferc1_pumped_storage_plants])
ferc1_all_plants = ferc1_respondents.merge(ferc1_all_plants, how='left', on='respondent_id')
ferc1_all_plants = ferc1_all_plants.replace('^$', np.nan, regex=True)
ferc1_all_plants = ferc1_all_plants.dropna(subset=['plant_name',])
ferc1_all_plants['report_year'] = ferc1_all_plants.report_year.astype(int)
ordered_for_cgosnell = ['respondent_id', 'respondent_name', 'plant_name',] #'report_year',\
                        #'kind_of_fuel',  'capacity_rating', 'plant_kind', 'tot_capacity']
ferc1_all_plants = ferc1_all_plants[ordered_for_cgosnell]
ferc1_all_plants = ferc1_all_plants.drop_duplicates(subset=['respondent_id','plant_name'])

In [ ]:
ferc1_all_plants.to_csv('../../results/id_mapping/ferc1_all_plants_2004-2015.csv', index=False)

In [ ]:
ferc1_respondents.to_csv('../../results/id_mapping/ferc1_all_respondents_2004-2015.csv', index=False)

In [ ]:
import os.path
map_eia923_ferc1_file = os.path.join(settings.PUDL_DIR,
                                         'results',
                                         'id_mapping',
                                         'mapping_eia923_ferc1.xlsx')

In [ ]:
plant_map = pd.read_excel(map_eia923_ferc1_file,'plants_output',
                          na_values='', keep_default_na=False,
                          converters={'plant_id':int,
                                      'plant_name':str,
                                      'respondent_id_ferc1':int,
                                      'respondent_name_ferc1':str,
                                      'plant_name_ferc1':str,
                                      'plant_id_eia923':int,
                                      'plant_name_eia923':str,
                                      'operator_name_eia923':str,
                                      'operator_id_eia923':int})

In [ ]:
plants_ferc1 = plant_map[['respondent_id_ferc1','respondent_name_ferc1','plant_name_ferc1']]
plants_ferc1 = plants_ferc1.drop_duplicates(['plant_name_ferc1','respondent_id_ferc1'])

In [ ]:
plants_ferc1.count()

In [ ]:
ferc1_all_plants.sample(1)

In [ ]:
plants_ferc1 = set([ tuple(line) for line in plants_ferc1.values.tolist()])
ferc1_all_plants= set([ tuple(line) for line in ferc1_all_plants.values.tolist()])

In [ ]:
plants_other_years = pd.DataFrame(list(plants_ferc1.difference(ferc1_all_plants)))

In [ ]:
plants_other_years.count()

In [ ]:
plants_other_years.sample(5)

In [ ]:
plants_other_years.to_csv('../../results/id_mapping/plants_ferc1_2004-2014.csv', index=False)